In [ ]:
import httpx

In [ ]:
def snomed_browser(term: str, limit: int = 100, semanticTag: str = "procedure") -> list[dict]:
    """
    Search SNOMED CT for concepts matching the given term.
    """
    base_url = "https://snowstorm.snomedtools.org/snowstorm/snomed-ct"
    edition = "MAIN"
    version = ""

    url = f"{base_url}/browser/{edition}/{version}/descriptions"

    params = {
        "term": term,
        "conceptActive": "true",
        "lang": "english",
        "limit": limit,
    }
    if semanticTag:
        params["semanticTag"] = semanticTag

    with httpx.Client() as client:
        response = client.get(url, params=params)
        response.raise_for_status()
        items = response.json().get("items", [])

    return items

In [ ]:
search_term = "colonoscopy"
matches = snomed_browser(search_term)

HTTPStatusError: Client error '429 Too Many Requests' for url 'https://snowstorm.snomedtools.org/snowstorm/snomed-ct/browser/MAIN//descriptions?term=colonoscopy&conceptActive=true&lang=english&limit=100&semanticTag=procedure'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/429